## 実践Python スクレイピングと可視化

今日お伝えしたことをスクレイピングと可視化をしながら学びます

### すること
1. `download` ディレクトリに格納している開店閉店情報をスクレイピング
1. データを前処理
1. 可視化

### しないこと
- html コンテンツのダウンロード
    - 時間がかかるし、みんながアクセスすると迷惑になるのでダウンロードしておきました。download ディレクトリに格納しています
    - ダウンロードのスクリプトは `src/download.py` に書いています

### 時間が出来たらやりたいこと
- 作った関数を `src` に移して実行

### memo

- jupyter notebook では cell の最後に記述したコードの返り値が表示されます


## ダウンロードした [開店閉店](https://kaiten-heiten.com/)情報htmlをスクレイピング

- リスト化されている開店閉店情報から、日付け、カテゴリー、店名、URLを取得する
- python で scraping する時のライブラリはだいたい `BeautifulSoup` を使う
    - [Beautiful Soup Documentation — Beautiful Soup 4.9.0 documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)



### スクレイピングの流れ

1. 一つのファイルをスクレイピングする関数を作る
1. その関数を、全ファイルに対して実行し、一つのデータにまとめる
1. データの前処理
1. データの可視化

In [ ]:
# cell の大きさを100％にする
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# 使いたいライブラリやモジュールをインポート
from bs4 import BeautifulSoup

### contents を 読み込む

まずは一つの html ファイルをスクレイピングしてみましょう

- 今日やったところ
    - [ ] 関数
    - [ ] メソッド

In [ ]:
# htmlファイルを開いて、中身を読み込む
# open関数：引数に渡されたファイルパスのファイルを読み込んでファイル風オブジェクトを返す関数
# .read() メソッド: ファイル風オブジェクトがファイルの中身(データ)を文字列で返すメソッド
html = open("../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_1_.html").read()
print(type(html))


### コンテンツを スクレイピングするために beautifulsoup オブジェクトに変換
- 今日やったところ
    - [ ] クラス
    - [ ] クラスのインスタンス化
    - [ ] クラスの初期値設定

In [ ]:
# BeautifulSoup クラス：　
#   初期値にスクレイピングしたいコンテンツと、パーサーを指定してインスタンス化すると、BeautifulSoupインスタンスオブジェクトを返す

soup = BeautifulSoup(
    html, # スクレイピングしたいコンテンツ
    'html.parser'  #パーサーの指定
)

print(type(soup))

### beautifulsoup オブジェクトのメソッドを使って情報を抽出
今回は、CSSの情報を元にデータを抽出

- 今日やったところ
    - [ ] オブジェクト
    - [ ] オブジェクトが持つメソッドを確認 dir

In [ ]:
# soup オブジェクトが持つアトリビュートを確認 
# 各メソッドなどの説明は https://www.crummy.com/software/BeautifulSoup/bs4/ をみてね
print(dir(soup))

1. `_category_kantou_koushinetsu_tokyo_page_1_.html`をブラウザで開いて、デベロッパーツールを開く
1. inspector ボタンを押してブラウザ上で必要な情報の詳細を得る。
1. お店1つずつの情報が `article` というタグにまとまってたのでこれを[css-selector](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors) をつかって取得する．
    - `.select()` メソッドを使う
    - 引数にはタグ名、クラス名、IDなどを渡す
    



In [ ]:
# .select()メソッド： soupオブジェクトが持つデータで、 article タグを持つデータをリストで返すメソッド
soup.select("article")

In [ ]:
# 最初の1つだけほしい場合は、リストのインデックス指定か、`.select_one()` メソッドを使う
soup.select("article")[0]
soup.select_one("article") 

In [ ]:
# 各要素は bs4.element.Tag オブジェクト
print(type(soup.select("article")[0]))


In [ ]:
# オブジェクトなんだからメソッド持ってるよね？と確認すると使えそうなやつがたくさん用意されている
print(dir(soup.select("article")[0]))


article タグの中を確認してみる

```html
<article class="list col-md-12 post-287548 post type-post status-publish format-standard has-post-thumbnail hentry category-coffie category-tokyo category-close category-kantou_koushinetsu category-restaurant">
    <a class="post_links" href="https://kaiten-heiten.com/ginza-renoir-ginza2chome/" title="【閉店】喫茶室ルノアール 銀座2丁目店">
        <div class="list-block">
            <div class="post_thumb" style="background-image: url('https://i2.wp.com/kaiten-heiten.com/wp-content/uploads/images/2021/09/004-6.jpg?fit=640%2C399&amp;ssl=1')"><span> </span></div>
            <div class="list-text">
                <span class="post_time"><i class="icon icon-clock"></i> 2021-09-01</span>
                <span class="post_cat"><i class="icon icon-folder"></i> コーヒーショップ, 東京, 閉店情報, 関東・甲信越, 飲食店</span> 
                <h3 class="list-title post_ttl">【閉店】喫茶室ルノアール 銀座2丁目店</h3>
            </div>
        </div>
    </a>
</article>
```
- `a` タグの中で
    - `title` アトリビュートで店名
    - `<span class="post_time">` で日付け
    - `<span class="post_cat">` でカテゴリ
- `class=` 属性でカテゴリーリスト
- このデータを持つ `bs4.element.Tag` オブジェクトをメソッドで操作していけば、ほしいデータが抽出出来る

In [ ]:
# .select_one() メソッドで `a` タグオブジェクトを取得
a = soup.select("article")[0].select_one("a")


In [ ]:
print(type(a))

print(dir(a))


In [ ]:
# タグオブジェクトのメソッド .get() で指定の属性情報を取得
a.get("title")


In [ ]:
# タグオブジェクトのメソッド .select_one() で タグとcssを指定し該当のオブジェクトを取得、その後テキスト部分だけ取得する .get_text() メソッドでテキストデータを取得
a.select_one("span.post_time").get_text()

In [ ]:
# 同上
a.select_one("span.post_cat").get_text()

次に class の中のカテゴリーを取っていきます
```html
<article class="list col-md-12 post-287548 post type-post status-publish format-standard has-post-thumbnail hentry category-coffie category-tokyo category-close category-kantou_koushinetsu category-restaurant">
```


In [ ]:
# soup オブジェクトの .get() メソッドをつかって class の属性情報をリストで取得
# class の中身全部がカテゴリーではない
soup.select("article")[0].get("class")

In [ ]:
# 「category-」　という文字列が入っていない文字列は除去したい
# 文字列オブジェクトが持つ .find() メソッドを使うと、引数に渡した文字列が入っているならばその位置を、入っていなければ-1を返す
'category-coffie'.find("category")

In [ ]:
'post-287548'.find("category")

In [ ]:
# これをcategoryだけのリストを作成
l = list()
for cat in soup.select("article")[0].get("class"):
    if cat.find("category") > -1:
        l.append(cat)
print(l)
        

- 今日やったところ
    - [ ] 内包表記（ちょっとバージョンアップ版）


In [ ]:
# 条件付きリスト内包表記
l = [cat for cat in soup.select("article")[0].get("class") if cat.find("category") > -1]
print(l)

In [ ]:
# いままでのをまとめて関数にする
# 返り値は辞書に入れる

def get_details(soup):
    """
    soup: soup.select_one("article") もしくは、soup.select("article")[0] を想定
    """
    soup_a = soup.select_one("a")
    name = soup_a.get("title")
    post_time = soup_a.select_one("span.post_time").get_text()
    url = soup_a.get("href")
    category = soup_a.select_one("span.post_cat").get_text()

    category_tag = [cat for cat in  soup.get("class") if cat.find("category") > -1]# 

    return {
        "name":name, 
        "post_time":post_time,
        "url":url,
        "category":category,
        "category_tag":category_tag}

In [ ]:
# テスト
get_details(soup.select("article")[0])

In [ ]:
# これを article タグのデータ全てに適用する
[get_details(article) for article in soup.select("article")]

In [ ]:
# html ファイルを引数にとってスクレイピングして辞書リストを返す関数を定義
def fetch_all(f):
    html = open(f).read()
    soup = BeautifulSoup(html, 'html.parser')
    return [get_details(article) for article in soup.select("article")]
    


In [ ]:
# test
fetch_all("../download/【閉店】/_category_kantou_koushinetsu_tokyo_page_1_.html")


## `download` ディレクトリに入っている全ファイルをスクレイピング

### ファイルリストを取得

`download` ホルダにダウンロードしている全htmlファイルに get_details関数を適用してデータを取得
- 今日やったところ
    - [ ] 組み込みクラスのインポートとインスタンス化


In [ ]:
# glob --- Unix 形式のパス名のパターン展開 https://docs.python.org/ja/3/library/glob.html

import glob 
# 【閉店】ディレクトリに入っているファイルをワイルドカードで指定。当てはまるフルパスをリストで返す
print(glob.glob("../download/【閉店】/*.html"))



In [ ]:
# このまますると、二重リストになる
[fetch_all(f) for f in glob.glob("../download/【閉店】/*.html")[:3]]

    


In [ ]:
# こういう時は flatten 
# itertools に入っている chain.from_iterable クラスにネストしたデータを渡すとイテレータオブジェクトを返す
import itertools
itertools.chain.from_iterable([fetch_all(f) for f in glob.glob("../download/【閉店】/*.html")[:3]])


In [ ]:
# イテレータオブジェクトは list 関数に渡すと、リストになる
list(itertools.chain.from_iterable([fetch_all(f) for f in glob.glob("../download/【閉店】/*.html")[:3]]))

In [ ]:
# ここまでを関数化
# html のファイルリストを渡したら辞書データのリストを返す関数を定義

def html_to_data(file_list):
    return list(itertools.chain.from_iterable([fetch_all(f) for f in file_list]))

In [ ]:
# test 
heiten_files = glob.glob("../download/【閉店】/*.html")
html_to_data(heiten_files)

## データの前処理
- pandasを使ってデータを前処理する
- pandas のかんたんなチュートリアルは こちらをどうぞ[10 minutes to pandas — pandas 1.3.2 documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html)
- いろいろ深く勉強したい人は [改訂版　Pythonユーザのための Jupyter［実践］入門](https://www.amazon.co.jp/dp/B08G1HYL9P) がオススメです




In [ ]:
import pandas as pd


### なぜデータを辞書型にしてリストに格納したのか？

pandas の DataFrame クラスは、同じデータキーワードを持つデータのリストを渡すと、キーワードをコラムにしてDataFrameで返します。辞書型にしてリストのデータを作った理由はここにあります。


In [ ]:
heitens = html_to_data(heiten_files)


In [ ]:
df = pd.DataFrame(heitens)

In [ ]:
# DataFrame オブジェクトの .head() メソッド：最初の五行だけ表示
df.head()

In [ ]:
# .info() メソッド。各コラムのデータ情報
df.info()

### 時系列データは、日時情報を index に持つのが吉

- 開店閉店情報は時系列情報です。よって日付け情報である `post_time` を時系列インデックスに変換していたほうがいろいろ幸せなことが多いと思います。
- `post_time` の型（⇑のDtype）は object になっています。DataFrameで object とは文字列のことです。
- これを Pandas の `to_datetime()` メソッドでDateTime型へ変換します。
- その後、DataFrame の `set_index()` メソッドで `post_time`をDataFrame のインデックスにします

In [ ]:
# datetime64 型へ変換して上書き
df["post_time"] = pd.to_datetime(df["post_time"])
df.info()

In [ ]:
# 見た目にはあまり変わらない
df.head()

In [ ]:
# post_time を このDataFrameのインデックスにします
# .set_index() ： DataFrame オブジェクトが持つ .set_index() メソッド。コラム名を渡してそのコラムをIndexにした新しいDataFrameを返す
# ただし、inplace=True オプションを渡すと、元のDataFrameのデータを書き換え（上書き）する

df.set_index("post_time", inplace=True)


In [ ]:
df.head()

In [ ]:
# .sort_index() メソッド： インデックスで並び替えした新しいDataFrameを返す
# ただし、inplace=True オプションを渡すと、元のDataFrameのデータを書き換え（上書き）する
df.sort_index(inplace=True)

In [ ]:
df.head()

In [ ]:
# これまでの処理を関数化します
def data_to_dataframe(data):
    """
    data は html_to_data で変換されたリストデータ
    """
    df = pd.DataFrame(data)
    df["post_time"] = pd.to_datetime(df["post_time"])
    df.set_index("post_time", inplace=True)
    df.sort_index(inplace=True)
    return df 
    
    

In [ ]:
# test
df_heiten = data_to_dataframe(heitens)
df_heiten.head()

## 可視化

- DataFrameが持つメソッドを駆使して可視化しましょう

In [ ]:
# .resample() メソッド：　DataFrameが持つIndex情報を元にデータをリサンプリングしてResamplerオブジェクトを返す
df_heiten.resample("M") # M は月ごとにデータをリサンプリング

In [ ]:
# Resampler オブジェクトのメソッドを確認
print(dir(df_heiten.resample("M")))

In [ ]:
# Resampler オブジェクト の .count() メソッド: リサンプリングデータの数をDataFrameで返す
# つまり、毎月閉店した数を得ることが出来る
df_heiten.resample("M").count()

In [ ]:
# .count() メソッドは、DataFrameを返すので、DataFrameオブジェクトのメソッドが使える
# .plot() で描画しましょう

# どのコラムも同じ数字なので代表で "name" だけで描画
df_heiten.resample("M").count()["name"].plot()

In [ ]:
# 年越せなかった店(2020/12月)と、頑張って越したけど非常事態宣言で耐えれなくなった店(2021/1,2月)、って感じですかね？
# ただ、3月以降は急に減ってますよね。
df_heiten.resample("M").count()["name"].plot(kind="bar")

[新型コロナ 東京都 4回目の緊急事態宣言の内容は | NHK](https://www.nhk.or.jp/shutoken/coronavirus/life_tokyo.html)
